In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import numpy as np
import torchvision

from torchvision.models import resnet18, ResNet18_Weights

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4
# batch_size = 9
# batch_size = 16

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [66]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.NLLLoss() # Negative Log-Likelihood Loss

optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9) # lr = [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00001]

# optimizer = optim.Adam(net.parameters(), lr=0.0001) #(net.parameters(), lr=0.0001, momentum=0.9)
# optimizer = optim.RMSprop(net.parameters(), lr=0.0001)
# optimizer = optim.Adadelta(net.parameters(), lr=0.0001)
# optimizer = optim.Adagrad(net.parameters(), lr=0.0001)
# optimizer = optim.Adamax(net.parameters(), lr=0.0001)
# optimizer = optim.AdamW(net.parameters(), lr=0.0001)
# optimizer = optim.ASGD(net.parameters(), lr=0.0001)
# optimizer = optim.Rprop(net.parameters(), lr=0.0001)

In [67]:
loss_record = []
for epoch in range(5):  # loop over the dataset multiple times, (2, 5, 10, 20)

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            loss_record.append(running_loss / 2000)
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.355
[1,  4000] loss: -0.109
[1,  6000] loss: -0.302
[1,  8000] loss: -0.501
[1, 10000] loss: -0.710
[1, 12000] loss: -0.876
[2,  2000] loss: -1.167
[2,  4000] loss: -1.349
[2,  6000] loss: -1.545
[2,  8000] loss: -1.758
[2, 10000] loss: -1.963
[2, 12000] loss: -2.180
[3,  2000] loss: -2.401
[3,  4000] loss: -2.620
[3,  6000] loss: -2.823
[3,  8000] loss: -3.006
[3, 10000] loss: -3.194
[3, 12000] loss: -3.402
[4,  2000] loss: -3.660
[4,  4000] loss: -3.877
[4,  6000] loss: -4.069
[4,  8000] loss: -4.246
[4, 10000] loss: -4.450
[4, 12000] loss: -4.655
[5,  2000] loss: -4.913
[5,  4000] loss: -5.099
[5,  6000] loss: -5.308
[5,  8000] loss: -5.503
[5, 10000] loss: -5.712
[5, 12000] loss: -5.922
Finished Training


In [68]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

acc = 100 * correct // total
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 10 %


In [69]:
import pandas as pd

results_dict = {'loss': loss_record, 'acc_percent': acc, 'lr': 0.0001, 'batch_size': 4, 'epochs': 5, 'exp': 'SGD_NNLoss'}
results_df = pd.DataFrame(results_dict)
results_df.to_csv('q12_SGD_NNLoss.csv')